In [1]:
import os
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from time import time
from PIL import Image

from keras import layers, models, optimizers
from keras import backend as K
K.set_image_data_format('channels_last')
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras import callbacks

from utils import combine_images
from utils import plot_log

from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
sys.path.append('./PatchyTools/')
from GraphConverter import GraphConverter
from DropboxLoader import DropboxLoader
from CapsuleParameters import CapsuleParameters
from CapsuleParameters import CapsuleTrainingParameters
from GraphClassifier import GraphClassifier


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Load data

In [2]:
    
# Getting the data:
dataset_name = 'MUTAG'
width = 18
receptive_field = 10
PatchyConverter = GraphConverter(dataset_name, width, receptive_field)
mutag_tensor = PatchyConverter.graphs_to_Patchy_tensor()
# plt.imshow(mutag_tensor[0,:,:,2])

# Getting the labels:
dropbox_loader = DropboxLoader(dataset_name)
mutag_labels = dropbox_loader.get_graph_label()
mutag_labels = np.array(mutag_labels.graph_label)

# Assigning the data:
x_train, x_test, y_train, y_test = train_test_split(mutag_tensor, mutag_labels, test_size=0.10)
data = ((x_train,y_train),(x_test, y_test))
input_shape = x_train.shape[1:]

MUTAG tensor exists, loading it from Dropbox


In [3]:
# Get the training Arguments
args = CapsuleTrainingParameters()

# Get the NN Parameters 
capsule_params= CapsuleParameters()
# First conv layer: 'filters', kernel_size)
conv_layer_name = 'conv_layer'
conv_layer_params = {}
conv_layer_params['filters'] = 256
conv_layer_params['kernel_size'] = 9
conv_layer_params['strides'] = [1, 1]
conv_layer_params['padding'] = 'VALID'
conv_layer_params['activation'] = 'relu'
conv_layer_params['name'] = 'conv1'
capsule_params.add_params(conv_layer_params,conv_layer_name)

# First Capsule Layer:
# [num_output_caps, caps_len,'filters',kernel_size,strides,padding]
caps_layer_name = 'caps_layer'
caps_layer_params = {}
caps_layer_params['filters'] = 256
caps_layer_params['kernel_size'] = 2
caps_layer_params['strides'] = [2,2]
caps_layer_params['padding'] = 'VALID'
caps_layer_params['padding'] = 'VALID'
# Parameters of the capsule:
caps_layer_params['n_channels'] = 32
caps_layer_params['dim_capsule'] = 8
caps_layer_params['name'] = 'caps_layer'
capsule_params.add_params(caps_layer_params,caps_layer_name)

# Digit Capsule Layer:
digit_layer_name = 'digitcaps_layer'
digit_layer_params = {}
digit_layer_params['n_channels'] = 10
digit_layer_params['dim_capsule'] = 16
digit_layer_params['name'] = 'digitcaps'
capsule_params.add_params(digit_layer_params,digit_layer_name )

# Capsule Decoder:
decoder_layer = 'decoder_layer'
decoder_params = {}
decoder_params['first_dense'] = 512
decoder_params['second_dense'] = 1024
decoder_params['name'] = 'decoder'
capsule_params.add_params(decoder_params,decoder_layer)



In [4]:
#args.data_augmentation = True
patchy_classifier = GraphClassifier(input_shape)
patchy_classifier.build_the_graph(capsule_params)
#patchy_classifier.train(data, args)


Instructions for updating:
dim is deprecated, use axis instead
time to generate the model: 0.6845037937164307


(<keras.engine.training.Model at 0x11ebfffd0>,
 <keras.engine.training.Model at 0x11ecb40b8>)

In [5]:
args.epochs = 2
patchy_classifier.train(data,args)

Train on 169 samples, validate on 19 samples
Epoch 1/2
169/169 [==============================] - 1s 4ms/step - loss: 0.8969 - capsnet_loss: 0.7990 - decoder_loss: 0.2499 - capsnet_acc: 0.4793 - val_loss: 0.6071 - val_capsnet_loss: 0.5093 - val_decoder_loss: 0.2494 - val_capsnet_acc: 0.6316

Epoch 00001: val_capsnet_acc improved from -inf to 0.63158, saving model to ./result/weights-01.h5
Epoch 2/2
169/169 [==============================] - 0s 1ms/step - loss: 0.4813 - capsnet_loss: 0.3837 - decoder_loss: 0.2492 - capsnet_acc: 0.6686 - val_loss: 0.3293 - val_capsnet_loss: 0.2321 - val_decoder_loss: 0.2478 - val_capsnet_acc: 0.6316

Epoch 00002: val_capsnet_acc did not improve from 0.63158
Trained model saved to './result/trained_model.h5'


In [6]:
model = patchy_classifier.train_model

In [45]:
patchy_classifier.results['time']=2342

In [46]:
patchy_classifier.results

epoch                  1.000000
capsnet_acc            0.668639
capsnet_loss           0.383662
decoder_loss           0.249166
loss                   0.481335
val_capsnet_acc        0.631579
val_capsnet_loss       0.232134
val_decoder_loss       0.247764
val_loss               0.329258
time                2342.000000
Name: 1, dtype: float64

'2018_09_28_15_8'

In [79]:
str(time_now.date())

'2018-09-28'

In [66]:
x.minute

4

In [81]:
for j in range(4):
    print(j)

0
1
2
3


In [82]:
j

3

In [30]:
print('shapes: ',x_test.shape,patchy_classifier.y_test.shape)
#model.evaluate([x_test,patchy_classifier.y_test],verbose=1)

shapes:  (19, 18, 10, 7) (19, 2)


In [33]:
vars(args)

{'batch_size': 100,
 'data_augmentation': False,
 'debug': False,
 'digit': 5,
 'epochs': 2,
 'lam_recon': 0.392,
 'lr': 0.001,
 'lr_decay': 0.9,
 'routing': 5,
 'save_dir': './result',
 'shift_fraction': 0.1,
 'testing': False,
 'weights': None}

In [34]:
list(vars(args).keys())

['epochs',
 'batch_size',
 'lr',
 'lr_decay',
 'lam_recon',
 'debug',
 'routing',
 'shift_fraction',
 'save_dir',
 'data_augmentation',
 'testing',
 'digit',
 'weights']

In [36]:
list_parameter_sets =[]
n_epoch_list = [50,100,150]
lr_list = [0.0005, 0.001, 0.005]
lr_decay_list = [0.25, 0.4, 0.75,1.5]
for n_epoch in n_epoch_list:
    for lr in lr_list:
        for lr_decay in lr_decay_list: 
            capsule_params = CapsuleTrainingParameters(epochs=n_epoch,lr=lr,lr_decay=lr_decay)
            list_parameter_sets.append(capsule_params)

In [ ]:
file_path = '/Users/marcelogutierrez/Projects/Gamma/capsuleSans/PatchyCapsules/result/log.csv'
df = pd.read_csv(file_path,index_col=0)

In [ ]:
pd.concat([test_acc, test_acc2],1)